 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Import" data-toc-modified-id="Import-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Import</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Common" data-toc-modified-id="Common-0.1.1"><span class="toc-item-num">0.1.1&nbsp;&nbsp;</span>Common</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Libs" data-toc-modified-id="Libs-0.1.2"><span class="toc-item-num">0.1.2&nbsp;&nbsp;</span>Libs</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Special" data-toc-modified-id="Special-0.1.3"><span class="toc-item-num">0.1.3&nbsp;&nbsp;</span>Special</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Load-longitudinal-retention" data-toc-modified-id="Load-longitudinal-retention-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>Load longitudinal retention</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Move" data-toc-modified-id="Move-0.2.1"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>Move</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Convert" data-toc-modified-id="Convert-0.2.2"><span class="toc-item-num">0.2.2&nbsp;&nbsp;</span>Convert</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Load-longitudinal-seq=1" data-toc-modified-id="Load-longitudinal-seq=1-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Load longitudinal seq=1</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Convert-to-parq" data-toc-modified-id="Convert-to-parq-0.3.1"><span class="toc-item-num">0.3.1&nbsp;&nbsp;</span>Convert to parq</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Load-longitudinal-seq=2" data-toc-modified-id="Load-longitudinal-seq=2-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>Load longitudinal seq=2</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Move-raw" data-toc-modified-id="Move-raw-0.4.1"><span class="toc-item-num">0.4.1&nbsp;&nbsp;</span>Move raw</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Convert-to-parq" data-toc-modified-id="Convert-to-parq-0.4.2"><span class="toc-item-num">0.4.2&nbsp;&nbsp;</span>Convert to parq</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#debug" data-toc-modified-id="debug-0.4.2.1"><span class="toc-item-num">0.4.2.1&nbsp;&nbsp;</span>debug</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#debug-end" data-toc-modified-id="debug-end-0.4.2.2"><span class="toc-item-num">0.4.2.2&nbsp;&nbsp;</span>debug end</a></span></li></ul></li></ul></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#Load-Onboarding-A/B" data-toc-modified-id="Load-Onboarding-A/B-0.5"><span class="toc-item-num">0.5&nbsp;&nbsp;</span>Load Onboarding A/B</a></span></li><li><span><a href="http://localhost:8888/notebooks/notebooks/io.ipynb#OOM" data-toc-modified-id="OOM-0.6"><span class="toc-item-num">0.6&nbsp;&nbsp;</span>OOM</a></span></li></ul></ul></div>

## Import

### Common

In [9]:
import boot_utes as bu; bu.reload(bu); from boot_utes import (reload, add_path, path)
add_path('../src/', '~/repos/myutils/')

9

In [10]:
from collections import defaultdict, Counter, OrderedDict
from functools import partial
from itertools import count
from operator import itemgetter as itg
from pprint import pprint
import os, sys, itertools as it, simplejson, time
from os.path import join
from glob import glob

### Libs

In [11]:
# from gensim.models import Word2Vec

import feather
import numpy as np
import numpy.random as nr
import pandas as pd
from pandas import DataFrame, Series
from pandas.compat import lmap, lrange, lfilter, lzip

import toolz.curried as z

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.mode.use_inf_as_na = True

# %mkdir cache
import joblib; mem = joblib.Memory(location='cache')
dbx = '/Users/wbeard/miniconda3/envs/tmpdecomp2/bin/databricks'

In [12]:
def norm_rows(df):
    return df.div(df.sum(axis=1), axis=0)

In [13]:
from os.path import join
import re

def get_spk_parq_fn(dir, ext='parquet'):
    s3parq_fs = !aws s3 ls $dir/
#     print(dir)
#     print(s3parq_fs)
    spk_parq_re = re.compile(r'part.+?\.{}'.format(ext))
    fn_all = spk_parq_re.findall(s3parq_fs[-1])
    if not len(fn_all) == 1:
        print('Uh oh: fn_all=', fn_all)
    [fn] = fn_all
    return fn

def show_s3_opts(*dir_elems):
    dir = join(*dir_elems)
    if not dir.endswith('/'):
        dir += '/'
    res = !aws s3 ls $dir
    return res
    
def mv_s3_pq(from_dir='', to_dir='data/pings', fn='pings_0125.parq', s3base=None, ext='parquet'):
    s3base = s3base or 's3://mozilla-databricks-telemetry-test/wbeard/'
    s3dir = join(s3base, from_dir, fn)
    
    try:
        pqn_ = get_spk_parq_fn(s3dir, ext=ext)
        print('Found pq file', pqn_)
    except IndexError:
        print('ERROR: File {} not found'.format(s3dir))
        opts = show_s3_opts(s3base, from_dir)
        print('\nPossible options:\n\t', '\n\t'.join(opts))
        return
        
    
    s3loc = join(s3base, from_dir, fn, pqn_)
    to_loc = join(to_dir, fn)
    
    
#     print(s3loc)
#     print(to_loc)
    !aws s3 cp $s3loc $to_loc
    return to_loc
    


### Special

In [5]:
# import utils.seq_utils as su; reload(su)
# import utils.events_utils as eu; reload(eu)
# import utils.feather_counter as fc; reload(fc)
# import utils.mutes as mt; reload(mt)

import myutils as mu
import functoolz as fz; reload(fz)
import numba_utils as nu; reload(nu)

mt.set_mem(mem=mem, json=simplejson)
su.set_memo(memoizer=mt.json_memo())
;;

NameError: name 'mt' is not defined

In [9]:
def move_new(dbx_dir, outdir=None, outdir_base='../data/raw/'):
    full_dbx_dir = join("dbfs:/wbeard/", dbx_dir)
    outdir = join(outdir_base, dbx_dir) if outdir is None else outdir
    dbx_fs = !$dbx fs ls $full_dbx_dir
    written = os.listdir(outdir)
    tobewritten = [f for f in dbx_fs if f not in written]
    for f in tobewritten:
        dbx_fn = join(full_dbx_dir, f)
        print('{} => {}'.format(dbx_fn, outdir))
        !$dbx fs cp $dbx_fn $outdir
    return tobewritten, written
    

def keep_moving(dbx_dir, outdir=None, outdir_base='../data/raw/', maxn=5):
    while 1:
        time.sleep(3)
        tobewritten, written = move_new(dbx_dir, outdir=outdir,
                                        outdir_base=outdir_base)

        if maxn and (len(tobewritten) + len(written) >= maxn):
            return
        print('.', end='')
        
        
vc = lambda x: Series(x).value_counts(normalize=0).sort_index()


In [12]:
!aws s3 sync 's3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_201801' data/p29_0202

download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/_SUCCESS to data/p26_8/_SUCCESS
download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/_committed_6840194297520358502 to data/p26_8/_committed_6840194297520358502
download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/_started_6840194297520358502 to data/p26_8/_started_6840194297520358502
download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/part-00002-tid-6840194297520358502-d54968ad-db37-4285-8f8c-e2e58808f762-0-c000.snappy.parquet to data/p26_8/part-00002-tid-6840194297520358502-d54968ad-db37-4285-8f8c-e2e58808f762-0-c000.snappy.parquet
download: s3://mozilla-databricks-telemetry-test/wbeard/crash_pings/p_20180126_20180128_eq_42.parq/part-00000-tid-6840194297520358502-d54968ad-db37-4285-8f8c-e2e58808f762-0-c000.snappy.parquet to data/p26_8/part-00000-tid-

## Win_proc

In [14]:
ls ../../win_proc/data/raw/

till_0204.pq


In [15]:
local_dir = '../../win_proc/data/raw/'
base = 's3://net-mozaws-prod-us-west-2-pipeline-analysis/wbeard/wl/'

In [16]:
# base = 'dbfs:/wbeard/apb/'
pqdir = 'till_0207.pq'
# pqdir = 'abc'

# , ext='json'
mv_s3_pq(from_dir='', to_dir=local_dir, fn=pqdir, s3base=base)

Found pq file part-00000-tid-6114354128941207063-a497a401-5843-4966-9646-3d2d6bae1b20-2389165-c000.snappy.parquet
download: s3://net-mozaws-prod-us-west-2-pipeline-analysis/wbeard/wl/till_0207.pq/part-00000-tid-6114354128941207063-a497a401-5843-4966-9646-3d2d6bae1b20-2389165-c000.snappy.parquet to ../data/raw/till_0207.pq


'../../win_proc/data/raw/till_0207.pq'

In [24]:
json_file = join(base, pqdir)
print(json_file)
!aws s3 cp "$json_file" "$local_dir"

s3://net-mozaws-prod-us-west-2-pipeline-analysis/wbeard/ipc/samp.json
fatal error: An error occurred (404) when calling the HeadObject operation: Key "wbeard/ipc/samp.json" does not exist


In [27]:
!aws s3 ls "$json_file/"

2018-11-28 12:23:24          0 _SUCCESS
2018-11-28 12:23:24        117 _committed_3179433506591924477
2018-11-28 12:23:23          0 _started_3179433506591924477
2018-11-28 12:23:24      51818 part-00000-tid-3179433506591924477-7c90a292-abf3-44e3-98b3-7213773c5a1d-10458612-c000.json
